# Como implementei

O ponto de partida foi uma classe `Turtle`:

In [ ]:
from jupyturtle import Turtle

alcides = Turtle()
alcides.forward(100)

Mas eu queria funções globais para controlar "a" tartaruga.
Qual tartaruga? Por hora, vamos controlar `alcides`:

In [ ]:
def lt(n):
    alcides.left(n)

lt(120)

In [ ]:
def fd(n):
    alcides.forward(n)

fd(50)

Isso fica chato bem rápido porque `Turtle` tem muitos métodos.

Não quero escrever uma função para cada um deles.

# Ideia: um decorador de métodos

Uma classe mais simples, com estado `self.ligado` e três métodos principais: `ligar`, `desligar`, e `clicar` (para alternar entre ligado e desligado automaticamente).

In [ ]:
class Botão():

    def __init__(self):
        self.ligado = False

    def __repr__(self):
        estado = 'LIGADO' if self.ligado else 'desligado'
        return f'<Botão {estado} (id={id(self):x})>'

    def ligar(self):
        self.ligado = True
        return self

    def desligar(self):
        self.ligado = False
        return self

    def alternar(self):
        self.ligado = not self.ligado
        return self
     

In [ ]:
b = Botão()
b

In [ ]:
b.ligar()

In [ ]:
b.alternar()

In [ ]:
b.alternar()

## O decorador que registra os comandos

In [ ]:
_comandos = []

def comando(metodo):
    _comandos.append(metodo.__name__)
    return metodo  # importante!!!

A classe decorada:

In [ ]:
class Botão():

    def __init__(self):
        self.ligado = False

    def __repr__(self):
        estado = 'LIGADO' if self.ligado else 'desligado'
        return f'<Botão {estado} (id={id(self):x})>'

    @comando
    def ligar(self):
        self.ligado = True
        return self

    @comando
    def desligar(self):
        self.ligado = False
        return self

    @comando
    def alternar(self):
        self.ligado = not self.ligado
        return self
     

Resultado do decorador:

In [ ]:
_comandos

## Função para construir um comando

Primeiro, vamos revisar o conceito de "método desvinculado"

### Métodos desvinculados

Métodos desvinculados (*unbound methods*) podem ser usados como função.

Normalmente usamos métodos vinculados, assim:

In [ ]:
s = "PyLadies"
s.upper()

Mas podemos obter o método desvinculado direto na classe `str`, e usar como uma função:

In [ ]:
str.upper(s)

Métodos desvinculados são úteis em contextos de programação imperativa ou funcional.

Por exemplo:

In [ ]:
frutas = ['abacaxi', 'banana', 'caqui', 'damasco']
list(map(str.upper, frutas))

Mas eu prefiro obter o mesmo resultado assim:

In [ ]:
[fruta.upper() for fruta in frutas]

### Voltando ao tema do Botão

Para cada comando,  precisamos obter um método desvinculado (*unbound method*) para usar como função:

In [ ]:
alt = getattr(Botão, 'alternar')  # obter método desvinculado (unbound)
alt

In [ ]:
b = Botão()
b

In [ ]:
alt(b)

In [ ]:
alt(b)

Agora vamos reunir essa lógica em uma função:

In [ ]:
_botão_principal = Botão()

def faz_comando(nome):
    método_desvinculado = getattr(Botão, nome)

    def cmd(*args):
        return método_desvinculado(_botão_principal, *args)

    cmd.__name__ = nome
    return cmd

alt = faz_comando('alternar')

In [ ]:
alt()

## Instalar comandos como funções globais

Veja como transformar uma mísera função anônima numa impressionante função global:

In [ ]:
globals()['dobro'] = lambda x: x * 2

dobro(21)

Agora já sabemos como instalar os comandos criados dinamicamente como funções globais:

In [ ]:
def instalar_comandos():
    for nome in _comandos:
        cmd = faz_comando(nome)
        nome_curto = nome[:3]
        globals()[nome_curto] = cmd

instalar_comandos()

In [ ]:
lig()

In [ ]:
des()

In [ ]:
alt()

## Conteúdo extra

### Notebooks

* [Exemplos do capítulo 4 de *Think Python, 3e*](ThinkPythonChap04.ipynb)
* [Galeria de exemplos](gallery.ipynb)
* [Referência de comandos](command-ref.ipynb)

### Código de `jupyturtle.py`

* [Comandos com apelidos](https://github.com/ramalho/jupyturtle/blob/d26dda0bb9f9bec383f393cfb1b1431e23e17030/src/jupyturtle/jupyturtle.py#L92): `forward` e `fd`

* [Mágica `%%turtle`](https://github.com/ramalho/jupyturtle/blob/d26dda0bb9f9bec383f393cfb1b1431e23e17030/src/jupyturtle/jupyturtle.py#L558)